# Stokes drifters

In [1]:
# import needed packages
import numpy as np


In [11]:
# define needed functions
def Buoyancy_drifter(d, h, m, rho_water):
    V = 0.25 * np.pi * d * d * h
    rho_drifter = m / V
    B = rho_drifter/ rho_water
    return B

def stokes_relaxation_time(d, viscosity_water, B):
    return (1 + 2 * B) * d * d / (36 * viscosity_water)


In [16]:
d = 0.24  # m diameter outer ring
d_in = 0.20 # m (estimated) dimater inner ring, needed to caluclate volume
h = 0.041  # m (heigth drifter)
m = 0.905 # kg (mass drifter)
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
viscosity_water = 1.41 * 10**(-3) # Ns/m^2 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
B = Buoyancy_drifter(d_in, h, m, rho_water)
tau = stokes_relaxation_time(d, viscosity_water, B)
print(f'B = {B:.2f}')
print(f'tau = {tau:.2f}')


B = 0.68
tau = 2.69
